##### Copyright 2025 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemma 3: Native tool-calling via Model Context Protocol (MCP)

<a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/examples/Gemma_3_Native_Tool_Calling_via_MCP.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" height=30/></a>

Gemma 3 is a highly capable open-weights model from Google DeepMind. One of its standout features is native support for function calling via standardized control tokens. 

This tutorial demonstrates how to integrate Gemma 3 with the **Model Context Protocol (MCP)**. MCP is an open standard that allows models to reach out to local or remote system environments to perform actions, search for data, or interact with files in a secure and standardized way.

In this notebook, you will learn:
1. How to define MCP tool logic.
2. How to map MCP tool definitions to Gemma 3's native function calling format.
3. How to implement an autonomous execution loop for agentic workflows.

## Setup

First, you install the necessary libraries. You use the `mcp` SDK for the protocol logic and `google-genai` for model interaction (if using hosted Gemma via Vertex AI) or standard Python libraries for local simulation.

In [ ]:
%pip install -q -U mcp httpx 'google-genai>=1.0.0'

## 1. Define MCP tool logic

The Model Context Protocol allows you to expose local functions to an LLM. Here, you define a simple `FileCreator` tool that allows Gemma to create files in the local environment.

In [ ]:
# @title MCP tool definitions and implementation

import json
from typing import Dict, Any, Optional

class LocalMCPGateway:
    """
    A simplified MCP Gateway that manages tool registration and execution.
    """
    def __init__(self):
        self.registry = {}

    def register_tool(self, name: str, description: str, schema: Dict[str, Any], func: Any):
        self.registry[name] = {
            "description": description,
            "schema": schema,
            "func": func
        }

    def get_tool_definitions(self) -> str:
        """
        Formats tools for Gemma 3's <start_function_declaration> block.
        """
        definitions = []
        for name, info in self.registry.items():
            def_str = f"declaration:{name}{json.dumps(info['schema'])}"
            definitions.append(def_str)
        return "\n".join(definitions)

    async def call_tool(self, name: str, arguments: Dict[str, Any]) -> str:
        """
        Executes a tool and returns the result encapsulated for Gemma.
        """
        if name not in self.registry:
            return f"Error: Tool {name} not found."
        
        try:
            result = await self.registry[name]["func"](**arguments)
            return str(result)
        except Exception as e: # Generic catch for tool execution errors reported back to model
            return f"Execution Error: {str(e)}"

async def create_local_file(filename: str, content: str) -> str:
    """
    Creates a file on the local filesystem.
    """
    try:
        with open(filename, 'w') as f:
            f.write(content)
        return f"Successfully created {filename}."
    except (IOError, OSError) as e:
        return f"Failed to create file: {str(e)}"

# Initialize and Register
gateway = LocalMCPGateway()
gateway.register_tool(
    name="create_file",
    description="Creates a new file with the specified content.",
    schema={
        "type": "object",
        "properties": {
            "filename": {"type": "string", "description": "The name of the file to create."},
            "content": {"type": "string", "description": "The content to write to the file."}
        },
        "required": ["filename", "content"]
    },
    func=create_local_file
)

## 2. Gemma 3 integration logic

Gemma 3 uses specialized tokens to handle tool-calling. You need to wrap the model interaction to parse these tokens and feed results back into the context.

In [ ]:
# @title Native token handling utilities

import re
import json
from typing import Optional, Dict, Any

GEMMA_SYSTEM_PROMPT = """
You are a model that can do function calling with the following functions:
<start_function_declaration>
{tool_definitions}
<end_function_declaration>

When you need to use a tool, use the following format:
<thought>
[Your reasoning here]
</thought>
<start_function_call>call:{tool_name}{{"arg1": "value1"}}<end_function_call>

You will then receive a response in this format:
<start_function_response>... result ...<end_function_response>
"""

def parse_tool_call(text: str) -> tuple[Optional[str], Optional[Dict[str, Any]]]:
    """
    Regex parser for Gemma 3's official <start_function_call> token.
    """
    match = re.search(r"<start_function_call>call:(\w+)(\{.*?\})<end_function_call>", text, re.DOTALL)
    if match:
        return match.group(1), json.loads(match.group(2))
    return None, None

## 3. The execution loop

In a real-world scenario, you would call a Gemma 3 API (like Vertex AI or a local Ollama instance). Here, you simulate the model's responses to demonstrate the multi-turn protocol flow.

In [ ]:
import asyncio

async def run_gemma_mcp_session(user_query: str):
    """
    Demonstrates the full lifecycle of an MCP-enabled Gemma 3 session.
    """
    print(f"USER: {user_query}\n")
    
    # 1. Initialize System Instruction
    tool_defs = gateway.get_tool_definitions()
    system_instr = GEMMA_SYSTEM_PROMPT.format(tool_definitions=tool_defs)
    print(f"SYSTEM INSTRUCTION:\n{system_instr}\n")
    
    # 2. Simulate Model Output (In reality, this would be an API call)
    # Gemma sees the query and decides to call the tool.
    simulated_model_output = """<thought>
I need to create a file as requested by the user. I should use the 'create_file' tool.
</thought>
<start_function_call>call:create_file{"filename": "hello.txt", "content": "Hello from the Gemma 3 MCP Gateway!"}<end_function_call>"""
    
    print(f"MODEL THOUGHTS & CALL:\n{simulated_model_output}\n")
    
    # 3. Parse and Execute Tool via MCP
    tool_name, tool_args = parse_tool_call(simulated_model_output)
    
    if tool_name:
        print(f"--- EXECUTING MCP TOOL: {tool_name} ---")
        result = await gateway.call_tool(tool_name, tool_args)
        print(f"RESULT: {result}\n")
        
        # 4. Feed Result Back to Model
        # The response_token would be sent back to the model to get the final response.
        response_token = f"<start_function_response>{result}<end_function_response>"
        print(f"FEEDING BACK TO MODEL:\n{response_token}\n")
        
        # 5. Simulate Final Model Response (In reality, this is another API call with history)
        final_output = "I have successfully created 'hello.txt' with the greeting you requested."
        print(f"MODEL FINAL ANSWER: {final_output}")
    else:
        print("No tool call detected.")

# Run the demonstration using asyncio.run() for robustness
try:
    asyncio.run(run_gemma_mcp_session("Create a file named 'hello.txt' with a greeting."))
except RuntimeError: # Handle case where loop is already running (e.g. in some notebook environments)
    import nest_asyncio
    nest_asyncio.apply()
    asyncio.run(run_gemma_mcp_session("Create a file named 'hello.txt' with a greeting."))
except Exception: # Fallback if nest_asyncio is not available
    await run_gemma_mcp_session("Create a file named 'hello.txt' with a greeting.")

## Conclusion

By using **native control tokens** and the **Model Context Protocol**, Gemma 3 becomes a powerful agent capable of interacting with its environment in a decoupled, standardized way. This architecture allows developers to build specialized toolsets that can be shared across any MCP-compliant application.